In [1]:
# import os

# os.chdir('/Documents/CBA_Stats')

import pandas as pd
import re
from cbastats import DBHelper
import datetime
import pytz
import unicodedata
# from cbastats import Scraper
from cbastats import ScraperMongo
from cbastats.Team import *
from cbastats.Player import *
#from multiprocessing import Pool
from tqdm.notebook import tqdm

# Connect Database

In [2]:
mongodbio =DBHelper.MongoDBHelper() 
client = mongodbio.create_connection(DBHelper.MONGODB_USERNAME,DBHelper.MONGODB_PWD,DBHelper.MONGODB_ENDPOINT)

db = client['cbaStats']

db.list_collection_names()

existing database ['cbaStats', 'admin', 'local']


['cbaAllGamesBoxScore',
 'cbaGamesStaging',
 'uniqueBoxScores',
 'seasonTeamStats',
 'seasonPlayerStats',
 'cbaGames']

In [3]:
coll_cbaGames = db['cbaGames']
coll_cbaGamesStaging=db['cbaGamesStaging']

# Scrape current schedule

In [4]:
sina_scraper = ScraperMongo.SinaScraper(ScraperMongo.SINA_SCHEDULE_BASE_URL,ScraperMongo.ENCODING, ScraperMongo.PARSER, ScraperMongo.HEADERS)

In [15]:
seasons_to_scrape = [key for key in sina_scraper.scraper_params['qleagueid']]
seasons_to_scrape

['20-21',
 '19-20',
 '18-19',
 '17-18',
 '16-17',
 '15-16',
 '14-15',
 '13-14',
 '12-13',
 '11-12',
 '10-11',
 '09-10',
 '08-09',
 '07-08',
 '06-07',
 '05-06']

In [17]:
scraped_schedule=[]
for season_to_scrape in tqdm_notebook(seasons_to_scrape):
    if season_to_scrape == '05-06':
        print('skip 05-06 season - 无上场时间数据')
        continue
    scraped_schedule.extend(sina_scraper.scrape_schedule(season=season_to_scrape))
    ScraperMongo.time.sleep(2)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


skip 05-06 season - 无上场时间数据



In [18]:
# Insert new games

result = mongodbio.insert_new_games(scraped_schedule,coll_cbaGames,coll_cbaGamesStaging)

----------clean up staging collection----------
cbaGamesStaging operation acknowledged!
0 records were deleted from cbaGamesStaging .
cbaGames has 0 docs.
cbaGamesStaging has 0 docs.
----------insert records into staging collection----------
cbaGamesStaging operation acknowledged!
5496 records were inserted into cbaGamesStaging.
----------checking what records to insert into production----------
----------insert records into production collection----------
cbaGames operation acknowledged!
5496 records were inserted into cbaGames.
cbaGamesStaging operation acknowledged!
5496 records were deleted from cbaGamesStaging .
cbaGames has 5496 docs.
cbaGamesStaging has 0 docs.


# Update Game Stats within each game

In [5]:
current_time = datetime.datetime.now()
timezone_est = pytz.timezone('US/Eastern')
timezone_utc = pytz.timezone('UTC')
current_time = timezone_utc.localize(current_time)
current_time

datetime.datetime(2021, 1, 16, 12, 14, 36, 688150, tzinfo=<UTC>)

In [6]:
five_hours = datetime.timedelta(hours=5)

In [7]:
schedule_to_scrape = list(mongodbio.select_records(coll_cbaGames,filter={'详细统计':'','日期':{"$lt":current_time-five_hours}}))

In [8]:
len(schedule_to_scrape)

5416

In [9]:
games_stats=[]
bad_scrapes=[]
for batch in tqdm(range(len(schedule_to_scrape)//100)):
    temp_schedule_to_scrape = schedule_to_scrape[batch*100:100*(batch+1)]
    temp_games_stats, temp_bad_scrapes = sina_scraper.scrape_games(temp_schedule_to_scrape,1)
    games_stats.append(temp_games_stats)
    bad_scrapes.append(temp_bad_scrapes)
    
# scrape rest of the games
temp_games_stats, temp_bad_scrapes = sina_scraper.scrape_games(temp_schedule_to_scrape,1)
games_stats.append(temp_games_stats)
bad_scrapes.append(temp_bad_scrapes)


  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [03:26<00:00,  2.06s/it]


In [10]:
len(bad_scrapes)

55

In [14]:
len(games_stats)

55

In [26]:
full_games_stats=pd.concat(games_stats, ignore_index=True)

In [28]:
# put game_stats into a dictionary
game_stats_dict={}
unique_gameids = list(full_games_stats['GameID_Sina'].value_counts().index)
for gameid in unique_gameids:
    game_stats_dict[gameid] = full_games_stats.loc[full_games_stats['GameID_Sina']==gameid].to_dict('record')

In [34]:
len(game_stats_dict)

5375

In [33]:
# update the stats in mongo
results=[]
for game_id, game_stats in game_stats_dict.items():
#     print(game_id)
    results.append(coll_cbaGames.update_one({'GameID_Sina':game_id},{"$set":{"详细统计":game_stats}}))

### Handle Bad Scrapes

In [39]:
list_badscrapes=[item for items in bad_scrapes for item in items]